In [ ]:
# Instructions for labelling images.



In [2]:
# First initialize datasets. These are used for training and evaluating the model.

from pathlib import Path
from random import sample

from fortuna import ObjectDetectionDataset
from fortuna.data import pedestrians_dirpath
import fortuna.torchutils.transforms as T

# Specify paths to images and masks.
image_paths = sorted((pedestrians_dirpath / "Images").rglob("*"))
mask_paths = sorted((pedestrians_dirpath / "Masks").rglob("*"))

# We're going to use a random subsample of the dataset to validate the model.
n = len(image_paths)
training_fraction = 0.9
training_ix = sample(range(n), int(training_fraction*n))
validation_ix = list(set(range(n)) - set(training_ix))

# Create ObjectDetectionDatasets for training and validation.
training_dataset = ObjectDetectionDataset(
    image_paths=[image_paths[j] for j in training_ix],
    mask_paths=[mask_paths[j] for j in training_ix],
    transforms=[T.RandomHorizontalFlip(0.5)]
)
validation_dataset = ObjectDetectionDataset(
    image_paths=[image_paths[j] for j in validation_ix],
    mask_paths=[mask_paths[j] for j in validation_ix]
)

In [3]:
# Initialize the object detection model.

from fortuna import ObjectDetectionModel

model = ObjectDetectionModel(n_classes=2)

In [4]:
# Fit the model.

model.fit(
    training_dataset,
    validation_dataset
)

Epoch: [0]  [ 0/77]  eta: 0:08:39  lr: 0.000014  loss: 3.5802 (3.5802)  loss_classifier: 0.5174 (0.5174)  loss_box_reg: 0.2021 (0.2021)  loss_mask: 2.2705 (2.2705)  loss_objectness: 0.5849 (0.5849)  loss_rpn_box_reg: 0.0053 (0.0053)  time: 6.7491  data: 0.0268


KeyboardInterrupt: 

In [ ]:
# Evaluate the detector.

model.evaluate(
    validation_dataset
)

In [ ]:
# Export model.

model.write("path_to_model_file.onnx") # Export as e.g. ONNX model.